In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() # you can find the intents file here:https://github.com/Sanket758/AI-Chatbot-with-Tensorflow
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

29 documents
6 classes ['feeling', 'goodbye', 'greeting', 'options', 'response', 'thanks']
50 unique lemmatized words ["'m", "'s", ',', 'alright', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'dobro', 'doing', 'feeling', 'fine', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'me', 'next', 'nice', 'offered', 'okay', 'provide', 'see', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'today', 'what', 'you']


In [8]:
# Create our training data
training = []
# Create an empty array for our output
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    
    # Lemmatize each word (create base word in an attempt to represent related words)
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Create our bag of words array with 1 if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Add bag and output row to the training set
    training.append([bag, output_row])

# Shuffle our features and turn into np.array
random.shuffle(training)

# Convert training data into numpy arrays
training = np.array(training, dtype=object)

# Create train and test lists: X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


In [9]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

# Define the learning rate decay schedule
initial_lr = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr, 
    decay_steps=100000,  # Adjust this according to your model and dataset
    decay_rate=0.96, 
    staircase=True
)

# Using SGD with the learning rate decay schedule
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 1.7965 - accuracy: 0.2759
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7559 - accuracy: 0.1724
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7419 - accuracy: 0.2414
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6677 - accuracy: 0.4138
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 1.6789 - accuracy: 0.4138
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6163 - accuracy: 0.4483
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5883 - accuracy: 0.3793
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5007 - accuracy: 0.4483
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4133 - accuracy: 0.6207
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 1.2893 - accuracy: 0.6552
Epoch 11/200
6/6 [===========

In [13]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [14]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [15]:
# Utility Methods
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [16]:
# This function returns the response corresponding to the predicted class
def get_response(predicted_class):
    # Loop through the intents and find the corresponding response
    for intent in intents['intents']:
        if intent['tag'] == predicted_class:
            return random.choice(intent['responses'])

# Now, update the predict_class function to return the sentence response
def predict_class(sentence, model):
    # Get the bag of words representation of the sentence
    p = bow(sentence, words, show_details=False)
    
    # Predict the class probabilities using the model
    res = model.predict(p.reshape(1, -1))[0]
    
    # Find the index of the highest probability
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    # Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    
    # Get the predicted intent tag with the highest probability
    predicted_class = classes[results[0][0]]
    print(predicted_class)
    
    # Get the response corresponding to that predicted class
    response = get_response(predicted_class)
    
    return response


In [24]:
# Test the prediction function
sentence = 'Im mad at you'
response = predict_class(sentence, model)
print(response)

1/1 [==============================] - 0s 66ms/step
feeling
I'm doing fine. How are you?
